<a href="https://colab.research.google.com/github/FelipeBorgesdosSantos/ACrashCourse-In-PySpark/blob/main/Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update # updating apt get repository
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # installing java development kit
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz # get the apache distribution from apache.org
!tar xf spark-3.1.1-bin-hadoop3.2.tgz # unzipping apache distribution
!pip install -q findspark # installing findspark, which adds pyspark to the system path when we execute this code
# setting home environment variables, so we now where to find the respective libraries
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
# initiallizing pyspark
import findspark
findspark.init()
# importing pyspark and starting a session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [498 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,259 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,283 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,155 kB]
Get:13 http:/

In [2]:
# Setting schema and importing:
from pyspark.sql.types import *
myschema = StructType([
      StructField('ip_address', StringType()),
      StructField('Country', StringType()),
      StructField('Domain Name', StringType()),
      StructField('Bytes_used', IntegerType())
])
df = spark.read.csv("challenge.csv", header=True, schema = myschema)
df.show()
df.dtypes

+---------------+--------------+-----------------+----------+
|     ip_address|       Country|      Domain Name|Bytes_used|
+---------------+--------------+-----------------+----------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|
| 119.239.207.13|         China|         youtu.be|        51|
|  68.69.217.210|         China|        adobe.com|        10|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|
|   211.13.10.68|     Indonesia|          hud.gov|        29|
|   239.80.21.97|      Suriname|       smh.com.au|       218|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|
| 127.242.24.138|         China| surveymonkey.com|       123|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|
|   237.54.11.63|         China|       amazon.com|        83|
| 252.141.157.25|         Japan|      cornell.edu|       374|
|185.220.128.248|       Belgium|       weebly.com|       389|
|   151.77.19.45|   Afghanistan|independent.co.uk|       282|
|  9.161

[('ip_address', 'string'),
 ('Country', 'string'),
 ('Domain Name', 'string'),
 ('Bytes_used', 'int')]

In [3]:
df.describe().show()

+-------+------------+-----------+-------------+------------------+
|summary|  ip_address|    Country|  Domain Name|        Bytes_used|
+-------+------------+-----------+-------------+------------------+
|  count|        1000|       1000|         1000|              1000|
|   mean|        null|       null|         null|           514.369|
| stddev|        null|       null|         null|287.66708047357156|
|    min|0.23.109.109|Afghanistan|123-reg.co.uk|                 1|
|    max|99.73.193.76|     Zambia|   zimbio.com|              1000|
+-------+------------+-----------+-------------+------------------+



In [8]:
# Country is Mexico?
from pyspark.sql.functions import *
df = df.withColumn('IsMexico', when(df.Country == 'Mexico', 'YES').otherwise('NO'))
df.show()

+---------------+--------------+-----------------+----------+--------+
|     ip_address|       Country|      Domain Name|Bytes_used|IsMexico|
+---------------+--------------+-----------------+----------+--------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|      NO|
| 119.239.207.13|         China|         youtu.be|        51|      NO|
|  68.69.217.210|         China|        adobe.com|        10|      NO|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|      NO|
|   211.13.10.68|     Indonesia|          hud.gov|        29|      NO|
|   239.80.21.97|      Suriname|       smh.com.au|       218|      NO|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|      NO|
| 127.242.24.138|         China| surveymonkey.com|       123|      NO|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|      NO|
|   237.54.11.63|         China|       amazon.com|        83|      NO|
| 252.141.157.25|         Japan|      cornell.edu|       374|      NO|
|185.2

In [20]:
# Bytes used grouped by IsMexico column:
import pyspark.sql.functions as sqlfunc
df1 = df.groupBy('IsMexico').agg(sqlfunc.sum('Bytes_used').alias('SumOfBytesUsed'))
df1.show()

+--------+--------------+
|IsMexico|SumOfBytesUsed|
+--------+--------------+
|     YES|          6293|
|      NO|        508076|
+--------+--------------+



In [19]:
# Counting distinct IP adress per country:
df2 = df.groupBy('Country').agg(sqlfunc.countDistinct('ip_address').alias('qtd_ips_distintos')).orderBy(desc('qtd_ips_distintos'))
df2.show()

+--------------+-----------------+
|       Country|qtd_ips_distintos|
+--------------+-----------------+
|         China|              172|
|     Indonesia|              114|
|   Philippines|               65|
|        Russia|               56|
|        Brazil|               35|
|        Poland|               31|
|        Sweden|               28|
|         Japan|               25|
|Czech Republic|               23|
|      Portugal|               23|
|        France|               21|
|          Peru|               19|
|      Colombia|               17|
| United States|               15|
|       Ukraine|               14|
|     Argentina|               14|
|        Mexico|               13|
|      Thailand|               12|
|        Canada|               11|
|       Nigeria|               11|
+--------------+-----------------+
only showing top 20 rows

